In [23]:
# Importing libraries
#If this is giving an error about scalebar, run: "pip install matplotlib-scalebar" in the terminal
import geopandas as gpd
from pyproj import CRS
import requests
import geojson
%run Final_assignment

# Fetching data from WFS using requests, in json format, using bounding box over the helsinki area
#r = requests.get("""http://lipas.cc.jyu.fi/geoserver/lipas/ows?service=wfs&version=2.0.0&request=GetFeature&typeNames=lipas:lipas_3110_uimahalli&bbox=361500.0001438780454919,6665250.0001345984637737,403750.0001343561452813,6698000.0001281434670091,EPSG:3067&outputFormat=json""")

#fetching football stadiums to limit the number of used files
r = requests.get("""http://lipas.cc.jyu.fi/geoserver/lipas/ows?service=wfs&version=2.0.0&request=GetFeature&typeNames=lipas:lipas_1350_jalkapallostadion&bbox=361500.0001438780454919,6665250.0001345984637737,403750.0001343561452813,6698000.0001281434670091,EPSG:3067&outputFormat=json""")


# Creating GeoDataFrame from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Data check
data.head()

,geometry,id,nimi_fi,nimi_se,nimi_en,sahkoposti,www,puhelinnumero,koulun_liikuntapaikka,vapaa_kaytto,...,pintamateriaali_lisatieto,katsomosta_katettua_hlo,tulospalvelu_aanentoisto,kenttien_lkm,yleiso_wc,tulostaulu,ottelukello,kevytkate,maalikamera,lisatieto_fi
0,POINT (373021.069 6677730.101),76159,Keskusurheilukenttä,Centralidrottsplan,None,None,https://www.kauniainen.fi/kulttuuri_ja_vapaa-a...,None,True,None,...,None,3500,true,None,None,true,None,None,None,"Tekonurmi, juoksuradat, yleisurheilupisteitä"
1,POINT (376923.201 6673282.758),82743,Tapiolan urheilupuisto / Jalkapallostadion,Hagalunds idrottspark / Fotbollsstadion,Tapiolan up:n nurmikenttä i,liikunta@espoo.fi,http://www.espoo.fi/fi-FI/Kulttuuri_ja_liikunt...,046 8771 352,True,False,...,Luonnonnurmi,1000,true,1,true,true,true,true,None,Tapiolan jalkapallostadion on Esport Hongan ko...
2,POINT (384793.501 6674139.697),87638,Telia 5G Areena (Töölön jalkapallostadion),Telia 5G Arena (Tölö fotbollsstadion),None,etunimi.sukunimi@5gareena.fi,http://hyvinvointi.hjk.fi/,None,None,None,...,None,10770,true,1,None,true,true,None,true,Helsinki stadion management Oy on HJK:n omista...
3,POINT (380366.469 6682542.462),87653,Myyrmäen jalkapallostadion,Myrbackas fotbollstadion,ISS Stadion,info@mup.fi,http://www.mup.fi/jalkapallostadion/#1,09 4770 450,None,None,...,Tekonurmi,2250,true,1,true,true,true,,,Pintamateriaali keinonurmi. Soveltuu lajeille ...


In [24]:
# Removing unnecessary attributes from lipas_data
lipas_data = data[["geometry","id","nimi_fi","nimi_se","tyyppikoodi","tyyppi_nimi_fi"]]

# Data check
lipas_data.head()

,geometry,id,nimi_fi,nimi_se,tyyppikoodi,tyyppi_nimi_fi
0,POINT (373021.069 6677730.101),76159,Keskusurheilukenttä,Centralidrottsplan,1350,Jalkapallostadion
1,POINT (376923.201 6673282.758),82743,Tapiolan urheilupuisto / Jalkapallostadion,Hagalunds idrottspark / Fotbollsstadion,1350,Jalkapallostadion
2,POINT (384793.501 6674139.697),87638,Telia 5G Areena (Töölön jalkapallostadion),Telia 5G Arena (Tölö fotbollsstadion),1350,Jalkapallostadion
3,POINT (380366.469 6682542.462),87653,Myyrmäen jalkapallostadion,Myrbackas fotbollstadion,1350,Jalkapallostadion


In [25]:
## Creating a list of YKR_IDs based on the location of sport facilities

# Set filepath for YKR grid and read the data
ykr_fp = r"data/MetropAccess_YKR_grid_EurefFIN.shp"
ykr_grid = gpd.read_file(ykr_fp)

# Defining crs for lipas_data (ykr_grid is already set)
lipas_data.crs = {'init':'epsg:3067'}

# Executing a spatial join to find out YKR-grid cells that have a sport facility inside them
lipas_join = gpd.sjoin(lipas_data, ykr_grid, how="inner", op="within")

# Collecting unique YKR_IDs into a list
ykr_list = lipas_join['YKR_ID'].unique().tolist()

# Data check
print(ykr_list)

[5936682, 5967140, 5961882, 5900202]


In [26]:
FileFinder(ykr_list)
TableJoiner(ykr_list)

Processing file data/HelsinkiTravelTimeMatrix2018/5936xxx/travel_times_to_ 5936682.txt. Progress: 1/4
Processing file data/HelsinkiTravelTimeMatrix2018/5967xxx/travel_times_to_ 5967140.txt. Progress: 2/4
Processing file data/HelsinkiTravelTimeMatrix2018/5961xxx/travel_times_to_ 5961882.txt. Progress: 3/4
Processing file data/HelsinkiTravelTimeMatrix2018/5900xxx/travel_times_to_ 5900202.txt. Progress: 4/4


ValueError: Invalid file path or buffer object type: <class 'int'>